In [28]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [29]:
import sys
sys.path.append("../")
import os

import re
import datetime
import pandas as pd
import pandas_market_calendars as mcal
from polygon import RESTClient
#from tqdm.notebook import tqdm
from progress.bar import Bar
import time
import pytz
eastern = 'US/Eastern'

In [30]:
key = "qRFktbNL6A_WzaFhqR80IllApsmjPY6a"

def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M')

def get_split_data_for_stock(stock):
    with RESTClient(key) as client:
        try:
            respex = client.reference_stock_splits(str(stock).upper())
            result = respex.__dict__['results']
            return result
        except:
            print("Error getting split data for "+ str(stock))
            return []

def get_dividend_data_for_stock(stock):
    with RESTClient(key) as client:
        try:
            respex = client.reference_stock_dividends(str(stock).upper())
            result = respex.__dict__['results']
            return result
        except:
            print("Error getting dividend data for "+ str(stock))
            return []
        
def extract_qualified_symbols(df, plvl, advlvl):
    qualified = df[(df.p_lvl == plvl) & (df.adv_lvl == advlvl)]
    return qualified

In [31]:
main_df = pd.read_csv('stocks-profile.csv')
q_sym_list = []
for i in range(5):
    q_sym = extract_qualified_symbols(main_df, i, 2)
    q_sym_list = q_sym_list + list(q_sym['symbol'])

In [32]:
splits_ = []
dividends_ = []

In [33]:
for i in tqdm(range(len(q_sym_list))):
    
    ticker = q_sym_list[i]
    
    sp = get_split_data_for_stock(ticker)
    div = get_dividend_data_for_stock(ticker)
    
    splits_.append({
        'symbol':ticker,
        'splits':sp
    })
    
    dividends_.append({
        'symbol':ticker,
        'dividends':div
    })
    
    time.sleep(0.1)

In [34]:
splits_df = pd.DataFrame(splits_)
divs_df = pd.DataFrame(dividends_)

In [80]:
def get_folder_list():    
    path = '.'
    directory_contents = os.listdir(path)

    r = re.compile(".*pv_adv_[0-9]_[0-9]")
    folder_list = list(filter(r.match, directory_contents)) # Read Note
    return folder_list

def get_all_files_in_folder(folder):
    return os.listdir(folder)

def extract_date_from_filename(filename):
    split_a = filename.split('trades-')
    split_b = split_a[1].split('.')
    return split_b[0]

In [81]:
def update_relevant_files(symbol, split_ex, payment, ratio):
    a = get_folder_list()
    
    for folder in tqdm(a, desc="folders"):
        all_files = get_all_files_in_folder(folder)
        
        for file in tqdm(all_files, desc="Folder Files"):
            file_date = extract_date_from_filename(file)
            file_df = pd.read_csv(folder+'/'+str(file))
            file_df.loc[file_df['symbol'] == symbol, 'trade_date'] = file_date
            
            if split_ex == file_date:
                
                sm_list = list(file_df['symbol'])
                
                if symbol in sm_list:
                    
                    file_df.loc[file_df['symbol'] == symbol, 'split_ex_date'] = split_ex
                    
                    file_df.loc[file_df['symbol'] == symbol, 'payment_date'] = payment
                    file_df.loc[file_df['symbol'] == symbol, 'ratio'] = ratio


                    print("file match - " + symbol + " - " + file_date)

                    file_df.to_csv(folder+'/'+file)

                    break
                else:
                    print("no match")
            
            file_df.to_csv(folder+'/'+file)


            
            
        

In [82]:
# update_relevant_files("SNDL", "2020-12-15",2,3)

In [148]:
symbols_with_dividends = divs_df[divs_df["dividends"].astype(bool)]

In [149]:
symbols_with_dividends.head()

,dividends,symbol
2,"[{'ticker': 'NOK', 'exDate': '2019-07-29', 'pa...",NOK
3,"[{'ticker': 'TOPS', 'exDate': '2006-04-12', 'p...",TOPS
11,"[{'ticker': 'IVR', 'exDate': '2021-01-11', 'pa...",IVR
15,"[{'ticker': 'RIG', 'exDate': '2015-08-21', 'pa...",RIG
16,"[{'ticker': 'MFA', 'exDate': '2021-03-30', 'pa...",MFA


In [83]:
symbols_with_splits = splits_df[splits_df["splits"].astype(bool)]

In [84]:
df = symbols_with_splits

In [85]:
df.head()

,splits,symbol
1,"[{'ticker': 'GNUS', 'exDate': '2016-11-09', 'p...",GNUS
2,"[{'ticker': 'NOK', 'exDate': '2000-04-11', 'pa...",NOK
3,"[{'ticker': 'TOPS', 'exDate': '2020-08-10', 'p...",TOPS
4,"[{'ticker': 'IBIO', 'exDate': '2018-06-11', 'p...",IBIO
6,"[{'ticker': 'JAGX', 'exDate': '2019-06-07', 'p...",JAGX


In [127]:
new_file_df = ouut.copy()

In [129]:

for index, row in tqdm(df.iterrows(), desc='Symbols', total=len(df)):
    
    for item in list(row['splits']):
        ex_date = item['exDate']

        payment_date = item['paymentDate']
        ratio = item['ratio']
        symbol = item['ticker']

        new_file_df.loc[(new_file_df['symbol'] == symbol) & (new_file_df['trade_date'] == ex_date), 'split_ex_date'] = ex_date
        new_file_df.loc[(new_file_df['symbol'] == symbol) & (new_file_df['trade_date'] == ex_date), 'split_ratio'] = ratio

In [ ]:
new_file_df[new_file_df['split_ratio'] > 0]

In [150]:
new_file_df_2 = new_file_df.copy()

In [151]:
for index, row in tqdm(symbols_with_dividends.iterrows(), desc='Symbols', total=len(symbols_with_dividends)):
    
    for item in list(row['dividends']):
        ex_date = item['exDate']

        payment_date = item['paymentDate']
        ratio = item['amount']
        symbol = item['ticker']

        new_file_df.loc[(new_file_df['symbol'] == symbol) & (new_file_df['trade_date'] == ex_date), 'div_ex_date'] = ex_date
        new_file_df.loc[(new_file_df['symbol'] == symbol) & (new_file_df['trade_date'] == ex_date), 'dividend_ratio'] = ratio

In [154]:
new_file_df[new_file_df['div_ex_date'] != "None"]

,Unnamed: 0,close_price,close_size,close_time,intraday_max_price,intraday_median_price,intraday_min_price,intraday_std_price,intraday_volume,open_price,...,pre_min_price,pre_std_price,pre_volume,symbol,trade_date,group,split_ex_date,div_ex_date,split_ratio,dividend_ratio
id,,,,,,,,,,,,,,,,,,,,,
4143245252151624499,127,5.04,455070,16:05:36.056297,6.4800,5.2500,4.8800,0.337939,3494112,5.70,...,5.0000,2.118443,1074009,AIV,2020-12-15,1,None,2020-12-15,1.0,34.868
2852561541928899254,33,3.70,192857,16:00:00.649914,4.0900,3.7150,3.6900,0.023298,130526,3.78,...,3.7399,0.019786,1938,NYMT,2020-12-16,1,None,2020-12-16,1.0,0.100
-3529872373839162679,31,3.13,1753945,16:00:02.519376,3.1700,3.1400,3.1100,0.013966,300400,3.14,...,3.0898,0.040042,4192,ABEV,2020-12-18,1,None,2020-12-18,1.0,0.069
1196459785474417410,188,4.66,136318,16:00:02.098777,4.7300,4.6500,4.5900,0.025180,98176,4.71,...,4.7000,0.022121,368,GGB,2020-12-22,1,None,2020-12-22,1.0,0.027
5225540497619197389,16,3.84,237473,16:00:02.107992,3.9700,3.8545,3.8300,0.026521,166384,3.95,...,3.8700,0.018489,3376,MFA,2020-12-29,1,None,2020-12-29,1.0,0.075
-359708871622343331,130,5.27,1030628,16:00:02.068257,5.3200,5.2800,5.2330,0.014226,247744,5.30,...,5.2900,0.011487,4144,BBD,2020-12-29,1,None,2020-12-29,1.0,0.061
1318293726746750618,11,3.35,283342,16:00:02.272003,3.4100,3.3500,3.3100,0.020810,242240,3.34,...,3.3000,0.022089,7232,IVR,2021-01-11,1,None,2021-01-11,1.0,0.080
-7131237639124090993,31,3.18,103044,16:00:02.065135,3.1850,3.1600,3.1000,0.019488,248208,3.11,...,3.0700,0.008849,1744,ABEV,2021-01-14,1,None,2021-01-14,1.0,0.015
7636497238489701184,129,3.89,107210,16:00:02.084569,4.0450,3.8500,3.7500,0.075847,180112,3.97,...,3.9400,0.028939,128,ENLC,2021-01-29,1,None,2021-01-29,1.0,0.094


In [72]:
import uuid

In [124]:
def update_all_files():
    a = get_folder_list()
    count = 0
    
    all_file_dfs = []
    
    for folder in tqdm(a, desc="folders"):
        all_files = get_all_files_in_folder(folder)
        count = count + 1
        
        for file in tqdm(all_files, desc="Folder Files"):
            file_date = extract_date_from_filename(file)
            file_df = pd.read_csv(folder+'/'+str(file))
            file_df['trade_date'] = file_date
            file_df['group'] = count
            file_df['id'] = file_df[['trade_date', 'symbol']].sum(axis=1).map(hash)            
            file_df['split_ex_date'] = "None"
            file_df['div_ex_date'] = "None"
            file_df['split_ratio'] = 1
            file_df['dividend_ratio'] = 0
            file_df.set_index("id", inplace=True)
            
            all_file_dfs.append(file_df)
            
    result_1 = pd.DataFrame().append(all_file_dfs)
    return result_1


            

In [125]:
ouut = update_all_files()

In [126]:
ouut.head()

,Unnamed: 0,close_price,close_size,close_time,intraday_max_price,intraday_median_price,intraday_min_price,intraday_std_price,intraday_volume,open_price,...,pre_min_price,pre_std_price,pre_volume,symbol,trade_date,group,split_ex_date,div_ex_date,split_ratio,dividend_ratio
id,,,,,,,,,,,,,,,,,,,,,
-4363773024735307697,0,0.5002,362823,16:00:01.213335,0.5125,0.4950,0.48,0.004855,1953632,0.5125,...,0.50,0.007189,133616,SNDL,2020-12-15,1,None,None,1,0
-3094733354799391054,1,1.6700,43982,16:00:01.227423,2.0000,1.7700,1.56,0.101874,1277248,1.9200,...,1.89,0.063828,113856,GNUS,2020-12-15,1,None,None,1,0
26083174918704691,2,4.1000,260074,16:00:02.042169,4.1500,4.0950,4.03,0.024144,387088,4.0500,...,4.00,0.013358,9008,NOK,2020-12-15,1,None,None,1,0
8889841163883510629,3,1.2900,4867,16:00:00.059332,1.3100,1.2650,1.24,0.016006,69488,1.2800,...,1.25,0.007524,2304,TOPS,2020-12-15,1,None,None,1,0
3918331231055451317,4,1.1300,66225,16:00:00.065081,1.1800,1.1289,1.12,0.011752,306208,1.1800,...,1.16,0.007394,10528,IBIO,2020-12-15,1,None,None,1,0


In [118]:
ouut[ouut['split_ex_date'] != "None"]

,Unnamed: 0,close_price,close_size,close_time,intraday_max_price,intraday_median_price,intraday_min_price,intraday_std_price,intraday_volume,open_price,...,pre_min_price,pre_std_price,pre_volume,symbol,trade_date,group,split_ex_date,div_ex_date,split_ratio,dividend_ratio
id,,,,,,,,,,,,,,,,,,,,,


In [158]:
all_etf_files = os.listdir('pv_adv_etf_data')

In [159]:
def update_all_etf_files():

    all_file_dfs = []    
    for file in tqdm(all_etf_files, desc="etf Files"):
        file_date = extract_date_from_filename(file)
        file_df = pd.read_csv("pv_adv_etf_data"+'/'+str(file))
        file_df['trade_date'] = file_date
        file_df['id'] = file_df[['trade_date', 'symbol']].sum(axis=1).map(hash)            
        file_df.set_index("id", inplace=True)

        all_file_dfs.append(file_df)
            
    result_1 = pd.DataFrame().append(all_file_dfs)
    return result_1

In [160]:
all_etf_files_df = update_all_etf_files()

In [162]:
all_etf_files_df.tail()

,Unnamed: 0,close_price,close_size,close_time,intraday_max_price,intraday_median_price,intraday_min_price,intraday_std_price,intraday_volume,open_price,...,post_min_price,post_std_price,post_volume,pre_max_price,pre_median_price,pre_min_price,pre_std_price,pre_volume,symbol,trade_date
id,,,,,,,,,,,,,,,,,,,,,
6026352170332198647,8,98.78,177412,16:00:00.032877,99.8833,98.545,98.14,0.171434,1297120,98.62,...,98.7000,0.105056,3456,98.80,98.50,98.45,0.079565,3696,XLI,2021-04-01
1336913473301179118,9,79.49,156394,16:00:00.022328,79.5450,79.100,78.45,0.289403,749152,78.93,...,79.2600,0.221010,736,79.11,78.95,78.73,0.094654,1184,XLB,2021-04-01
7607800608286401390,10,40.12,129487,16:00:00.196495,40.1200,39.650,39.47,0.150655,306976,39.68,...,40.0000,0.050712,256,39.70,39.50,39.40,0.100821,256,XLRE,2021-04-01
-6070077879132789308,11,135.48,275717,16:00:00.022336,135.4900,135.000,132.81,0.276598,944176,134.26,...,135.0943,0.109052,3344,134.44,134.20,132.81,0.471539,2560,XLK,2021-04-01
-4987016536567793848,12,63.98,1200732,16:00:00.043356,64.0900,63.800,63.55,0.118779,930121,63.94,...,63.8500,0.058233,612,64.18,64.05,63.90,0.047796,1819,XLU,2021-04-01


In [165]:
new_new_file = new_file_df.copy()

In [166]:
for index, row in tqdm(all_etf_files_df.iterrows(), desc='Symbols', total=len(all_etf_files_df)):

    for colname in all_etf_files_df.columns:
        
        new_new_file.loc[(new_new_file['trade_date'] == row['trade_date']), row['symbol']+'_'+colname] = row[colname]



In [168]:
list(new_new_file.columns)

['Unnamed: 0',
 'close_price',
 'close_size',
 'close_time',
 'intraday_max_price',
 'intraday_median_price',
 'intraday_min_price',
 'intraday_std_price',
 'intraday_volume',
 'open_price',
 'open_size',
 'open_time',
 'post_max_price',
 'post_median_price',
 'post_min_price',
 'post_std_price',
 'post_volume',
 'pre_max_price',
 'pre_median_price',
 'pre_min_price',
 'pre_std_price',
 'pre_volume',
 'symbol',
 'trade_date',
 'group',
 'split_ex_date',
 'div_ex_date',
 'split_ratio',
 'dividend_ratio',
 'SPY_Unnamed: 0',
 'SPY_close_price',
 'SPY_close_size',
 'SPY_close_time',
 'SPY_intraday_max_price',
 'SPY_intraday_median_price',
 'SPY_intraday_min_price',
 'SPY_intraday_std_price',
 'SPY_intraday_volume',
 'SPY_open_price',
 'SPY_open_size',
 'SPY_open_time',
 'SPY_post_max_price',
 'SPY_post_median_price',
 'SPY_post_min_price',
 'SPY_post_std_price',
 'SPY_post_volume',
 'SPY_pre_max_price',
 'SPY_pre_median_price',
 'SPY_pre_min_price',
 'SPY_pre_std_price',
 'SPY_pre_volume',

In [169]:
new_new_file.head()

,Unnamed: 0,close_price,close_size,close_time,intraday_max_price,intraday_median_price,intraday_min_price,intraday_std_price,intraday_volume,open_price,...,XLU_post_min_price,XLU_post_std_price,XLU_post_volume,XLU_pre_max_price,XLU_pre_median_price,XLU_pre_min_price,XLU_pre_std_price,XLU_pre_volume,XLU_symbol,XLU_trade_date
id,,,,,,,,,,,,,,,,,,,,,
-4363773024735307697,0,0.5002,362823,16:00:01.213335,0.5125,0.4950,0.48,0.004855,1953632,0.5125,...,63.0,0.039796,576.0,62.5,61.92,61.92,0.152788,736.0,XLU,2020-12-15
-3094733354799391054,1,1.6700,43982,16:00:01.227423,2.0000,1.7700,1.56,0.101874,1277248,1.9200,...,63.0,0.039796,576.0,62.5,61.92,61.92,0.152788,736.0,XLU,2020-12-15
26083174918704691,2,4.1000,260074,16:00:02.042169,4.1500,4.0950,4.03,0.024144,387088,4.0500,...,63.0,0.039796,576.0,62.5,61.92,61.92,0.152788,736.0,XLU,2020-12-15
8889841163883510629,3,1.2900,4867,16:00:00.059332,1.3100,1.2650,1.24,0.016006,69488,1.2800,...,63.0,0.039796,576.0,62.5,61.92,61.92,0.152788,736.0,XLU,2020-12-15
3918331231055451317,4,1.1300,66225,16:00:00.065081,1.1800,1.1289,1.12,0.011752,306208,1.1800,...,63.0,0.039796,576.0,62.5,61.92,61.92,0.152788,736.0,XLU,2020-12-15


In [170]:
new_new_file.to_csv("all_combined.csv")

In [49]:
res = str(splits_data['splits'][1]).strip('][').split(', ')


In [50]:
res

["{'ticker': 'GNUS'",
 "'exDate': '2016-11-09'",
 "'paymentDate': '2016-11-09'",
 "'declaredDate': '2016-11-09'",
 "'ratio': 3",
 "'tofactor': 3",
 "'forfactor': 1}"]